In [6]:
# ==============================================================================
# SECTION 1: IMPORTS & AUTHENTICATION
# ==============================================================================

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from huggingface_hub import notebook_login

print("\n🔑 Setting up Hugging Face authentication...")
notebook_login()

c:\Users\G2\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(



🔑 Setting up Hugging Face authentication...


In [7]:
# ==============================================================================
# SECTION 2: VERIFY HARDWARE SETUP
# ==============================================================================
print("\n🔍 Verifying GPU setup...")

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    vram_gb = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    
    print(f"✅ GPU is available!")
    print(f"   Device: {gpu_name}")
    print(f"   VRAM: {vram_gb:.2f} GB")
    
    if torch.cuda.is_bf16_supported():
        print("   bfloat16 is supported on this GPU (Good for training).")
    else:
        print("   bfloat16 is NOT supported. Will use float16 instead.")
else:
    print("❌ GPU not available. The code will run on CPU, which will be very slow for AI tasks.")

print("\n🎉 Environment setup is complete!")


🔍 Verifying GPU setup...
❌ GPU not available. The code will run on CPU, which will be very slow for AI tasks.

🎉 Environment setup is complete!


In [2]:
# ==============================================================================
# SECTION 3: PERSONALITY DATA SYNTHESIS WORKFLOW
# ==============================================================================
# ส่วนนี้คือ "โรงงาน AI" ที่จะแปลงค่าบุคลิกภาพไปเป็นข้อมูลสำหรับ Fine-tune
# ------------------------------------------------------------------------------

# 3.1 คลังเทมเพลต (The "Ingredients" Library)
# เราสร้างคลังเก็บ "ชิ้นส่วน" บทสนทนาและความคิดไว้ล่วงหน้า
# โดยแบ่งตามบุคลิกภาพ (เช่น agreeableness) และระดับ (high/low)

template_library = {
    "agreeableness": {
        "high": [
            "Dialogue: ต้องขออภัยในความไม่สะดวกด้วยจริงๆ นะคะ {customer_problem} ค่ะ เข้าใจเลยค่ะ เดี๋ยวทางเราจะรีบ {solution} ให้นะคะ!",
            "Dialogue: ไม่เป็นไรเลยค่ะ! ด้วยความยินดีค่ะ ทางเราจัดการให้ได้แน่นอนค่ะ"
        ],
        "low": [
            "Dialogue: เรื่อง {customer_problem} เหรอคะ ปกติก็ไม่เคยมีปัญหานะคะ แต่จะ {solution} ให้แล้วกันค่ะ",
            "Dialogue: ก็ตามนั้นแหละค่ะ"
        ]
    },
    "neuroticism": {
        "high": [
            "Thought: (แย่แล้ว ลูกค้าเริ่มเสียงดัง ทำยังไงดี จะจัดการได้ไหมเนี่ย กลัวจัง)",
            "Thought: (เราต้องทำอะไรสักอย่าง แต่ไม่รู้จะเริ่มตรงไหนก่อนดี ทุกอย่างดูวุ่นวายไปหมด)"
        ],
        "low": [
            "Thought: (โอเค มีปัญหาเกิดขึ้น ตั้งสติ แล้วแก้ไขไปทีละขั้นตอน)",
            "Thought: (สถานการณ์นี้จัดการได้ แค่ทำตามที่เคยฝึกมาก็พอ)"
        ]
    }
    # TODO: เพิ่มเทมเพลตสำหรับบุคลิกภาพด้านอื่นๆ (Openness, Conscientiousness, Extraversion)
}


# 3.2 โปรไฟล์บุคลิกภาพ (The "Recipe" from a Candidate)
# นี่คือผลคะแนนสมมติที่เราจะใช้ทดสอบระบบ
# คะแนนจะอยู่ในช่วง 0.0 (ต่ำมาก) ถึง 1.0 (สูงมาก)

candidate_profile_A = {
    "name": "Somsri (The Ideal Employee)",
    "scores": {
        "agreeableness": 0.92, # สูงมาก
        "conscientiousness": 0.88,
        "extraversion": 0.55,
        "neuroticism": 0.21,   # ต่ำมาก
        "openness": 0.75
    }
}


# 3.3 ฟังก์ชันสร้างข้อมูล (The "Robot Chef" Algorithm)
# โปรแกรมที่จะอ่าน "สูตรอาหาร" แล้วไปหยิบ "วัตถุดิบ" มาประกอบร่าง

def generate_training_data(profile_scores, templates):
    """
    ฟังก์ชันสำหรับสร้างชุดข้อมูล Fine-tune จากโปรไฟล์บุคลิกภาพและคลังเทมเพลต
    """
    
    # กำหนดเกณฑ์คะแนน สูง-ต่ำ
    HIGH_THRESHOLD = 0.7
    LOW_THRESHOLD = 0.3
    
    final_dataset = []
    
    # วนลูปเช็คบุคลิกภาพแต่ละด้าน
    for trait, score in profile_scores.items():
        
        level = None
        if score >= HIGH_THRESHOLD:
            level = "high"
        elif score <= LOW_THRESHOLD:
            level = "low"
        
        # ถ้าคะแนนมีค่าโดดเด่น (สูงหรือต่ำ)
        if level and trait in templates:
            # ดึงเทมเพลตทั้งหมดที่ตรงกับบุคลิกนั้นๆ
            trait_templates = templates[trait][level]
            
            # นำเทมเพลตมาสร้างเป็นข้อมูลตัวอย่าง
            for template in trait_templates:
                # ในโปรเจคจริง ส่วนนี้จะซับซ้อนกว่านี้ โดยจะมีการสุ่มสถานการณ์ต่างๆ
                populated_example = template.format(
                    customer_problem="เรื่องรสชาติไอศกรีม",
                    solution="เปลี่ยนถ้วยใหม่"
                )
                final_dataset.append(populated_example)
                
    return final_dataset


# 3.4 ทดลองรันฟังก์ชัน
# ลองรันฟังก์ชันเพื่อสร้างชุดข้อมูลสำหรับ Candidate A

print(f"กำลังสร้างชุดข้อมูลสำหรับ '{candidate_profile_A['name']}'...")
print("-" * 30)

generated_data = generate_training_data(candidate_profile_A['scores'], template_library)

# แสดงผลข้อมูลที่สร้างได้
for i, example in enumerate(generated_data):
    print(f"ตัวอย่างที่ {i+1}: {example}")

print("-" * 30)
print(f"สร้างชุดข้อมูลสำเร็จ! ทั้งหมด {len(generated_data)} ตัวอย่าง")

กำลังสร้างชุดข้อมูลสำหรับ 'Somsri (The Ideal Employee)'...
------------------------------
ตัวอย่างที่ 1: Dialogue: ต้องขออภัยในความไม่สะดวกด้วยจริงๆ นะคะ เรื่องรสชาติไอศกรีม ค่ะ เข้าใจเลยค่ะ เดี๋ยวทางเราจะรีบ เปลี่ยนถ้วยใหม่ ให้นะคะ!
ตัวอย่างที่ 2: Dialogue: ไม่เป็นไรเลยค่ะ! ด้วยความยินดีค่ะ ทางเราจัดการให้ได้แน่นอนค่ะ
ตัวอย่างที่ 3: Thought: (โอเค มีปัญหาเกิดขึ้น ตั้งสติ แล้วแก้ไขไปทีละขั้นตอน)
ตัวอย่างที่ 4: Thought: (สถานการณ์นี้จัดการได้ แค่ทำตามที่เคยฝึกมาก็พอ)
------------------------------
สร้างชุดข้อมูลสำเร็จ! ทั้งหมด 4 ตัวอย่าง
